In [1]:
!pip install peft

In [2]:
!pip install trl

In [4]:
import json
import os

def force_string_content(input_path, output_path):
    fixed_lines = []
    with open(input_path, 'r') as f:
        for line in f:
            data = json.loads(line)
            for msg in data['messages']:
                # If content is a dict or list, make it a JSON string
                if isinstance(msg.get("content"), (dict, list)):
                    msg["content"] = json.dumps(msg["content"])
                # Ensure tool_call_id is a string too
                if "tool_call_id" in msg:
                    msg["tool_call_id"] = str(msg["tool_call_id"])
            fixed_lines.append(json.dumps(data))
    
    with open(output_path, 'w') as f:
        f.write("\n".join(fixed_lines))
os.makedirs("llama_format_stringified", exist_ok=True)
force_string_content("llama_format/train.jsonl", "llama_format_stringified/train.jsonl")
force_string_content("llama_format/validation.jsonl", "llama_format_stringified/validation.jsonl")
force_string_content("llama_format/test.jsonl", "llama_format_stringified/test.jsonl")

In [ ]:
import torch
import os
import json
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
from trl import SFTConfig, SFTTrainer

device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

model_id = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Updated template
tokenizer.chat_template = """
{%- set system_message = messages[0]['content'] if messages[0]['role'] == 'system' else '' -%}
<|start_header_id|>system<|end_header_id|>\n\n{{ system_message }}<|eot_id|>
{%- for message in messages %}
    {%- if message.role == 'user' %}
        <|start_header_id|>user<|end_header_id|>\n\n{{ message.content }}<|eot_id|>
    {%- elif 'tool_calls' in message %}
        <|start_header_id|>assistant<|end_header_id|>\n\n
        {%- for tool_call in message.tool_calls %}
            {"name": "{{ tool_call.function.name }}", "parameters": {{ tool_call.function.arguments | tojson }}}
            {%- if not loop.last %}\n{% endif %}
        {%- endfor %}<|eot_id|>
    {%- elif message.role == 'assistant' %}
        <|start_header_id|>assistant<|end_header_id|>\n\n{{ message.content }}<|eot_id|>
    {%- endif %}
{%- endfor %}
{%- if add_generation_prompt %}<|start_header_id|>assistant<|end_header_id|>\n\n{% endif %}
"""


raw_dataset = load_dataset("json", data_files="llama_format_stringified/train.jsonl", split="train")
train_test_valid = raw_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_valid["train"]
eval_dataset = train_test_valid["test"]

print(f"Final Counts:")
print(f"  - Training samples:   {len(train_dataset)}")
print(f"  - Validation samples: {len(eval_dataset)}")

/Users/kohli1/Documents/RP_Work/fine-tuning/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: mps


ConnectError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1010)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map={"": device}
)

`torch_dtype` is deprecated! Use `dtype` instead!
Loading weights: 100%|██████████| 254/254 [00:16<00:00, 15.02it/s, Materializing param=model.norm.weight]                              


In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

sft_config = SFTConfig(
    output_dir="./llama3.2-finetuned-mps",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    optim="adamw_torch",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    bf16=True,
    report_to="none",
    dataset_text_field="text",
    packing=False
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config, 
    args=sft_config,
    processing_class=tokenizer
)

print("Starting training on MPS...")
trainer.train()

trainer.save_model("./llama3.2-final-adapter")
tokenizer.save_pretrained("./llama3.2-final-adapter")
print("All set! Your adapter is saved in ./llama3.2-final-adapter")

NameError: name 'train_dataset' is not defined